#Создание чат бота

Медицинская вопросно-ответную система, датасет для дообучения модели https://huggingface.co/datasets/blinoff/medical_qa_ru_data, из датасета взято 5000 строк для ускорения обучения

In [1]:
PATH_ =  "/content/drive/MyDrive/NLP/Course Project/"

# Дообучим модель

In [2]:
!pip install -q transformers
! wget https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv
! pip install corus

--2024-02-12 06:01:49--  https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/blinoff/medical_qa_ru_data/6aca4fb6dae499745c9ca1dfad8a0ade809573daf228c923fdf846ba4a639b50?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27medical_qa_ru_data.csv%3B+filename%3D%22medical_qa_ru_data.csv%22%3B&response-content-type=text%2Fcsv&Expires=1707976909&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzk3NjkwOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9ibGlub2ZmL21lZGljYWxfcWFfcnVfZGF0YS82YWNhNGZiNmRhZTQ5OTc0NWM5Y2ExZGZhZDhhMGFkZTgwOTU3M2RhZjIyOGM5MjNmZGY4NDZiYTRhNjM5YjUwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9za

In [4]:
!pip install -q transformers

In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 4.3 MB/s eta 0:00:00


In [6]:
#подключаем гугл диск
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
import pandas as pd
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

In [7]:
device = torch.device("cuda")

In [ ]:
data = pd.read_csv('medical_qa_ru_data.csv')
data

,date,categ,theme,desc,ans,spec10
0,"8 Октября 2017, 11:55",Оториноларингология,Применение Ларипронта.,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,Отоларинголог
1,"20 Февраля 2019, 13:24",Акушерство,Беременность,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",NaN
2,"17 Марта 2015, 18:31",Другое,гинекология,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,NaN
3,"13 Января 2019, 19:38",Терапия,Занятия спорта после сдачи крови,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",Терапевт
4,"28 Ноября 2017, 21:58",Другое,Таблетки,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,NaN
...,...,...,...,...,...,...
190330,"31 Июля 2014, 11:31",Офтальмология,Отлетела в глаз пластмасса,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю...",Офтальмолог
190331,"16 Марта 2014, 21:52",Другое,здравствуйте скажите пожалуста где я могу посм...,NaN,"В ""консультанте""",NaN
190332,"22 Ноября 2018, 11:02",Общая хирургия,палец,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",Хирург
190333,"2 Ноября 2018, 21:19",Эндокринология,Эндокринолог прописал пить сиофор 500 при саха...,NaN,Такие препараты принимаются постоянно;\nЗдравс...,NaN


In [ ]:
data.ans = data.ans.apply(lambda x: re.split(r';\n', str(x))[0])
data.ans[10]

'При развитии массивного кровотечения есть'

In [ ]:
data = data[['desc', 'ans']]
data.dropna(subset=['desc', 'ans'], inplace=True)
data['class'] = 1
data

<ipython-input-15-8b2c5c659da8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2.dropna(subset=['desc', 'ans'], inplace=True)
<ipython-input-15-8b2c5c659da8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2['class'] = 1


,desc,ans,class
0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,1
1,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",1
2,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,1
3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...",Можно.,1
4,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,1
...,...,...,...
190328,"Не курю и не пью. Веду здоровый образ жизни, м...",В первую очередь вам необходимо обследовать пе...,1
190329,"Здравствуйте, очень прошу консультации. В лево...",Начинать следует с терапевтических методов. Ес...,1
190330,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид",1
190332,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",1


In [ ]:
# т.к. модель очень долго обучаеться возьмем только 5000 строк
data = data.sample(5000)
data

,desc,ans,class
29197,Мрт Шейного отдела:\n \n\n ...,У вас явления распространенного остеохондроза ...,1
32684,"Здравствуйте, несколько дней назад сына укусил...","Здравствуйте , обработайте Мирамистин+фенистил",1
90241,Добрый день! Подскажите пожалуйста моей дочери...,"Назначенное лечение адекватное,насчет Египта -...",1
145159,Здравствуйте подскажите пожалуйста что это мож...,Возможно начало ОРВИ.,1
154218,Добрый вечер!у меня на теле переодически появл...,Вероятнее всего имеет место отрубевидный лишай...,1
...,...,...,...
175980,под грудью красные пятна с признаками шелушени...,Обратитесь к дерматологу - нужно видеть эти пя...,1
153778,Здравствуйте! Мне 19 лет.Подскажите пожалуйста...,Проверялся ли гормональный фон перед приемом о...,1
43651,"Сегодня мне резали десну,у меня флюс,прошло 6 ...","Дома вы вряд ли что сделаете,нужно туго затамп...",1
145081,"Здравствуйте, доктор. Помогите мне пожалуйста....",nan,1


In [ ]:
#сохраняем датасет
data.to_csv(PATH_ + 'qa_medical.csv', encoding='utf-8', index=False)

Разобьем датасет на вопросы и ответы, тренировочную и тестовую выборки:

In [ ]:
train_q, test_q, train_ans, test_ans = train_test_split(data.desc,
                                                        data.ans,
                                                        test_size=.2,
                                                        random_state=42)

Скачаем модель GPT от сбера:

In [ ]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Функции для простого препроцессинга (удаление непечатных знаков) и сборки датасета для обучения. При сборке датасета вопрос помечаем как '\nx:', а ответ - как '\ny:'

In [ ]:
def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [ ]:
train_data = build_data(train_q, train_ans)
test_data = build_data(test_q, test_ans)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

Токенизируем тренировочный и тестовый датасеты:

In [ ]:
train_encodings = tokenizer(train_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

Класс для создания torch-датасета:

In [ ]:
class QAMedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = QAMedicalDataset(train_encodings)
test_dataset = QAMedicalDataset(test_encodings)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Зададим аргуметы для дообучения модели:

In [ ]:
training_args = TrainingArguments(
    output_dir="./qa_medical", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Обучим модель:

In [ ]:
trainer.train()

<ipython-input-27-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,4.580800
1000,3.788900
1500,3.607600
2000,3.568100
2500,3.562700
3000,3.835300


<ipython-input-27-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-27-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-27-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=3000, training_loss=3.8239251708984376, metrics={'train_runtime': 1902.6743, 'train_samples_per_second': 6.307, 'train_steps_per_second': 1.577, 'total_flos': 3135504384000000.0, 'train_loss': 3.8239251708984376, 'epoch': 3.0})

Сохраним модель на диск:

In [ ]:
tokenizer.save_pretrained(PATH_ + 'tokenizer_qa_medical_gen')
model.save_pretrained(PATH_ + 'model_qa_medical_gen')

Загружаю модель с гугл диска, модель долго обучаеться, бот писался не один день и каждый день обучать модель накладно по времени

In [9]:
tokenizer_mq = AutoTokenizer.from_pretrained(PATH_ + 'tokenizer_qa_medical_gen')
model_mq = AutoModelForCausalLM.from_pretrained(PATH_ + 'model_qa_medical_gen')

Функция для ведения диалога.

In [10]:
def respond_to_med_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer_mq(prefix, return_tensors='pt')
    tokens = {k: v.to(model_mq.device) for k, v in tokens.items()}
    end_token_id = tokenizer_mq.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model_mq.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size+128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer_mq.eos_token_id
    )
    decoded = tokenizer_mq.decode(output[0])
    result = re.findall(r'\ny:(.+)', decoded)[-1]
    return result.strip()

Протестируем генератор:

In [10]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_med_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'Стоп':
        break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
у меня болит голова
Выполните МРТ сосудов головного мозга,сделайте рентген шейного отдела. Сдайте общий анализ крови на гормоны щитовидной железы и прогеографию поясничного отдела позвоночника, сдайте кровь на хгцр-аденергиома гипофибидроскопию, лимфоузохориацитоэнтероциты в норме, узи щитовидной желез внутренней секреции, УЗИ щитовидной железы, брюшной полости, эндокринолога, биохимический аорты, обратитесь к неврологу для назначения лечения остеохондроз шейного отдела позвоночника, гепатол
я простыл, у меня заложен нос как лечить?
Здравствуйте! Обратитесь к ЛОР врачу он не назначает лечение только за консультативному лечению по переписке вы можете назначить вам нужно исключить патологию, так как это может повлиять на онкологии или что то другое. У вас нет возможности выполнить самолечение. Но при наличии противопоказаний могут быть проблемы с желудочно-кишечным трактом (печени, желчегонный рефлюкс, гастрит, дисбактериоз, диарея, холецистит, поджелу

In [ ]:
#!pip install torch wikipedia torchvision torchaudio

In [12]:
!pip install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 90.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu121
    Uninstalling torchvision-0.16.0+cu121:
      Successfully uninstalled torchvision-0.16.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.1.0+cu121
    Uninstalling torchaudio-2.1.0+cu121:
      Successfully uninstalled torchaudio-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0,

In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model_name_or_path = "radm/rugpt3medium-tathagata"
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)#.to(DEVICE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [14]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size+128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=end_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

#"Бот-вики"

In [15]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=45336573641ff43ebf3bb8361f06c622e9fc96c9048c4a3294e21c5d68483719
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [16]:
import wikipedia
import re
wikipedia.set_lang("ru")

In [17]:
# Чистим текст статьи из Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        wikitext=ny.content[:2000]
        wikimas=wikitext.split('.')
        wikimas = wikimas[:-1]
        wikitext2 = ''
        for x in wikimas:
            if not('==' in x):
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return 'В энциклопедии нет информации об этом'

In [17]:
print(getwiki('Тесла'))

Нико́ла Те́сла  — американский инженер и физик-электроник, изобретатель в области электротехники и радиотехники, внёс существенный вклад в развитие электроинженерии. Наиболее известный разработкой современной системы электроснабжения переменного тока. После победы в «Войне токов» и демонстрации радио стал одним из самых известных личностей в истории науки ХХ века. 
По национальности — серб, родился в Австрийской империи, вырос в Австро-Венгрии, в последующие годы в основном работал во Франции и США. В 1891 году получил гражданство США.

Широко известен благодаря своему вкладу в создание устройств, работающих на переменном токе, многофазных систем, синхронного генератора и асинхронного электродвигателя, позволивших совершить так называемый второй этап промышленной революции. Также он известен как сторонник теории о существовании эфира — благодаря своим многочисленным опытам и экспериментам, имевшим целью показать наличие эфира как особой формы материи, поддающейся использованию в техник

In [18]:
print(getwiki('Тут тебе не там'))

Де́сять за́поведей  — предписания, десять основных законов, которые, согласно Пятикнижию, были даны Моисею самим Богом, в присутствии сынов Израиля, на горе Синай на пятидесятый день после исхода из Египта .
Десять заповедей содержатся в Пятикнижии в двух мало отличающихся друг от друга версиях . В другом месте  часть заповедей воспроизведена в форме комментария, вложенного в уста Всевышнего, при этом не комментируются морально-нравственные нормы, но формулируются предписания в религиозно-культовой области. Согласно еврейской традиции, вариант, содержащийся в 20-й главе книги Исход, был на первых, разбитых скрижалях, а вариант Второзакония — на вторых.
Обстановка, в которой Бог дал Моисею и сынам Израиля Десять заповедей, описана в Библии. Синай стоял в огне, окутанный густым дымом; земля дрожала; гремел гром; блистали молнии; и в шуме разбушевавшейся стихии, покрывая его, раздавался голос Божий, произносивший заповеди (Исх. 19:1 и след. Затем сам Господь начертал «Десять слов» на двух

In [19]:
print(getwiki('Создать чат бот'))

В энциклопедии нет информации об этом


# Bot

In [18]:
!pip install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.15.4-py3-none-any.whl size=231937 sha256=eff1321747addca2d192bdfac9fb388d7fe9415db4dfa9d0734b0f95448b6b56
  Stored in directory: /root/.cache/pip/wheels/37/50/a0/68519c75209358c42758e5ed2d3c9da9afda4d20a3ac393070
Successfully built pytelegrambotapi


In [19]:
history_dialog = []

def babbler(text):
    global history_dialog

    history_dialog.append(text)
    history_dialog.append(respond_to_dialog(history_dialog[-10:]))
    if len(history_dialog) > 10: history_dialog = history_dialog[-10:]

    return history_dialog[-1]

In [26]:
import telebot
from telebot import types

# Создаем экземпляр бота, https://t.me/NLPCourseProject_bot
# Если запускать не на графическом ускарителе(Cuda) медицинский бот тормозит жутко
bot = telebot.TeleBot("6351581982:AAHRfPAItxEh2_nr-YGR_mofjUYyl3dcr4U")

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Привет")
    markup.add(btn1)
    bot.send_message(message.chat.id, "👋 Привет! Я бот-помошник!", reply_markup=markup)

Flag = 0
@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    global Flag
    # Если нажали на одну из кнопок
    if call.data.lower() == 'поболтаем':
        Flag = 1
        bot.send_message(call.message.chat.id, 'Привет! О чём желаете поговорить? :)')
    if call.data.lower() == 'вики-бот':
        Flag = 2
        bot.send_message(call.message.chat.id, 'Какую информацию Вам найти в интернете?')
    if call.data.lower() == 'медицинский-бот':
        Flag = 3
        bot.send_message(call.message.chat.id, 'Что Вас беспокоит, напишите проблему, а медицинский чат-бот постараеться Вам помочь')

# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])
def handle_text(message):
    global Flag

    if message.text.lower() == 'привет': Flag = 0


    if Flag == 0:
        keyboard = types.InlineKeyboardMarkup()
        # добавляем кнопки на экран
        key_1 = types.InlineKeyboardButton(text='Поболтаем', callback_data='поболтаем')
        keyboard.add(key_1)
        key_2 = types.InlineKeyboardButton(text='Вики-бот', callback_data='вики-бот')
        keyboard.add(key_2)
        key_3 = types.InlineKeyboardButton(text='Медицинский бот', callback_data='медицинский-бот')
        keyboard.add(key_3)
        bot.send_message(message.chat.id, text='Сделайте выбор', reply_markup=keyboard)

    elif Flag == 1:
        bot.send_message(message.chat.id, babbler(message.text))
    elif Flag == 2:
        bot.send_message(message.chat.id, getwiki(message.text))
    elif Flag == 3:
        history = [message.text]
        result = respond_to_med_dialog(history[-10:])
        bot.send_message(message.chat.id, result)
        history.append(result)


# Запускаем бота
bot.polling(none_stop=True, interval=0)